# Simulation of a single chain

In [1]:
import unittest
import feasst as fst
print(fst.version())

def calc_rg2(mc):
    conf = mc.system().configuration()
    rg2_sum = 0.0
    for i in range(conf.particle(0).num_sites() - 1):
        pos = conf.particle(0).site(i).position()
        for j in range(i+1, conf.particle(0).num_sites()):
            rg2_sum += pos.squared_distance(conf.particle(0).site(j).position())
    rg2 = rg2_sum / (conf.particle(0).num_sites()**2)
    return rg2

mc = fst.MonteCarlo()
mc.set(fst.MakeRandomMT19937(fst.args({"seed": "time"})))
mc.add(fst.Configuration(fst.MakeDomain(fst.args({"cubic_box_length": "40",
                                                  "periodic0": "False",
                                                  "periodic1": "False",
                                                  "periodic2": "False"})),
                         fst.args({"particle_type": fst.install_dir() + "/plugin/chain/forcefield/chain5.fstprt"})))
mc.get_system().get_configuration().set_model_param("cutoff", 0, 20)
mc.add(fst.MakePotential(fst.MakeLennardJones(),
                         fst.MakeVisitModelIntra(fst.args({"cutoff": "1"}))))
mc.set(fst.MakeThermoParams(fst.args({"beta": "1", "chemical_potential0": "1"})))
mc.set(fst.MakeMetropolis())
fst.SeekNumParticles(1).with_trial_add().run(mc)
mc.add(fst.MakeTrialPivot(fst.args({"weight": "1", "tunable_param": "20"})))
mc.add(fst.MakeTrialCrankshaft(fst.args({"weight": "1", "tunable_param": "20"})))
#mc.add(fst.MakeTrialReptate(fst.args({"weight": "1", "max_length": "1", "particle_type": "0"})))
mc.add(fst.MakeTrialGrowLinear(fst.MakeTrialComputeMove(),
                               fst.args({"weight": "1", "particle_type": "0", "num_steps": "3"})))
steps_per = str(int(1e4))
mc.add(fst.MakeLogAndMovie(fst.args({"steps_per": steps_per, "file_name": "chain"})))
mc.add(fst.MakeEnergy(fst.args({"steps_per_write": steps_per, "file_name": "chain_en.txt"})))
mc.add(fst.MakeCheckEnergyAndTune(fst.args({"steps_per": steps_per, "tolerance": str(1e-10)})))
print('Potential energy of initial configuration:', mc.criteria().current_energy())
rg2 = fst.Accumulator()
mc.attempt(int(1e3)) # equilibrate
for n in range(int(1e5)):
    mc.attempt()
    rg2.accumulate(calc_rg2(mc))
            
class TestChain5RG(unittest.TestCase):
    def test_rg_en(self):
        print(rg2.str())
        self.assertAlmostEqual(0.8806, rg2.average(), delta=0.01)
        self.assertAlmostEqual(-2.050,
            fst.SeekAnalyze().reference("Energy", mc).accumulator().average(),
            delta=0.02)

v0.16.0-30-gbf11101ab9-hwh/doc2
Potential energy of initial configuration: -0.19650552006989855


In [2]:
%time
unittest.main(argv=[''], verbosity=2, exit=False)

test_rg_en (__main__.TestChain5RG) ... 

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs
average,stdev,block_stdev,n,moment0,moment1,
0.88224112185913728,0.23397367129135313,0,100000,88224.112185913723,83309.252852003468,


ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!